In [4]:
import os
from osgeo import gdal

# Définir les chemins des fichiers et des dossiers
band_path = "../../data/images"
band_numbers = ['2', '3', '4', '5', '6', '7', '8', '8A', '11', '12']
band_date = [
    "_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B",
    "_20220405-105855-542_L2A_T31TCJ_C_V3-0_FRE_B",
    "_20220714-105903-040_L2A_T31TCJ_C_V3-0_FRE_B",
    "_20220922-105859-863_L2A_T31TCJ_C_V3-1_FRE_B",
    "_20221111-105858-090_L2A_T31TCJ_C_V3-1_FRE_B",
    "_20230219-105857-687_L2A_T31TCJ_C_V3-1_FRE_B"
]  # Liste des dates pour chaque image

# Générer les chemins complets pour chaque bande
band_files = [f"{band_path}/SENTINEL2B{date}{band_number}.tif" for date, band_number in zip(band_date, band_numbers)]

# Répertoire de sortie
out_path = "../results/img_pretraitees"
mask_file = os.path.join("masque_foret.tif")  # Fichier de masque

# Vérifier et créer le répertoire de sortie s'il n'existe pas
if not os.path.exists(out_path):
    os.makedirs(out_path)

# Paramètres de sortie
output_file = os.path.join(out_path, "Serie_temp_S2_allbands.tif")
projection = "EPSG:2154"
pixel_size = 10  # Résolution spatiale en mètres
nodata_value = 0

# Fonction pour reprojeter et rééchantillonner une bande
def reproject_and_resample(band_file, output_file, pixel_size, projection):
    # Ouvrir le fichier de la bande comme un dataset GDAL
    band_ds = gdal.Open(band_file)
    if band_ds is None:
        raise Exception(f"Erreur lors de l'ouverture du fichier {band_file}")

    # Reprojection et rééchantillonnage
    gdal.Warp(
        output_file,
        band_ds,
        xRes=pixel_size,
        yRes=pixel_size,
        dstSRS=projection,
        dstNodata=nodata_value
    )
    band_ds = None  # Fermer le dataset après l'opération

# Étape 1 : Reprojeter et rééchantillonner toutes les bandes
reprojected_bands = []
for i, band_file in enumerate(band_files):
    reprojected_file = os.path.join(out_path, f"band_{i+1}_reprojected.tif")
    reproject_and_resample(band_file, reprojected_file, pixel_size, projection)
    reprojected_bands.append(reprojected_file)

# Étape 2 : Appliquer le masque forêt à chaque bande
masked_bands = []
for i, band_file in enumerate(reprojected_bands):
    masked_file = os.path.join(out_path, f"band_{i+1}_masked.tif")
    gdal_calc_command = (
        f"gdal_calc.py -A {band_file} -B {mask_file} --outfile={masked_file} "
        f"--calc=\"A*B\" --NoDataValue={nodata_value} --quiet"
    )
    os.system(gdal_calc_command)
    masked_bands.append(masked_file)

# Étape 3 : Fusionner toutes les bandes dans un seul fichier
# Utilisation de gdal_merge.py pour fusionner les bandes
merge_command = (
    f"gdal_merge.py -separate -o {output_file} " + " ".join(masked_bands) + f" -a_nodata {nodata_value}"
)
os.system(merge_command)

# Vérifier la taille du fichier et convertir en uint8 si nécessaire
output_uint8_file = output_file.replace(".tif", "_uint8.tif")
if os.path.getsize(output_file) > 10 * 1024 * 1024 * 1024:  # Si le fichier est plus grand que 10 Go
    dynamic_convert_command = (
        f"otbcli_DynamicConvert -in {output_file} -out {output_uint8_file} uint8"
    )
    os.system(dynamic_convert_command)
    print(f"Fichier converti en uint8 : {output_uint8_file}")
else:
    print(f"Fichier final généré : {output_file}")


Could not open file "masque_foret.tif": No such file or directory
Could not open file "masque_foret.tif": No such file or directory
Could not open file "masque_foret.tif": No such file or directory
Could not open file "masque_foret.tif": No such file or directory
Could not open file "masque_foret.tif": No such file or directory
Could not open file "masque_foret.tif": No such file or directory
Traceback (most recent call last):
  File "/opt/conda/bin/gdal_merge.py", line 12, in <module>
    sys.exit(main(sys.argv))
             ^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/osgeo_utils/gdal_merge.py", line 657, in main
    return gdal_merge(argv)
           ^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/osgeo_utils/gdal_merge.py", line 397, in gdal_merge
    return _gdal_merge(argv=argv)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/osgeo_utils/gdal_merge.py", line 530, in _gdal_merge
    file_infos = names_to_fileinf

FileNotFoundError: [Errno 2] No such file or directory: '../results/img_pretraitees/Serie_temp_S2_allbands.tif'